# Análise de Jogos Populares da Steam
---

## Importações

In [95]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

import requests
import json



---

## Coletando Dados e Armazenando em Arquivo CSV

In [98]:
# Configurações da janela do Selenium
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

# inicializar navegador
driver = webdriver.Chrome(options=options)

### Coletando IDs dos Jogos

In [99]:
year = 2019
all_ids = []

while year <= 2024:
    # os links de 2019, 2020 e 2021+ são diferentes, então:
    if year == 2019:
        game_id_url = f"https://store.steampowered.com/sale/{year}_most_played/"
    elif year == 2020:
        game_id_url = f"https://store.steampowered.com/sale/BestOf{year}?tab=1"
    else:
        game_id_url = f"https://store.steampowered.com/sale/BestOf{year}?tab=3"
        
    driver.get(game_id_url)

    time.sleep(5)

    # Encontrando IDs
    soup = BeautifulSoup(driver.page_source, "html.parser")

    a_tags = soup.find_all("a", class_="Focusable")
    a_tags = a_tags[:50]

    ids_year = []        
    for tag in a_tags:
        # 35 chars antes do id
        id = ''
        for num in (tag['href'][35:]):
            if num == "/":
                ids_year.append(id)
                break
            id += num
            
    all_ids.append(ids_year)
    year += 1
    
# liberando memória
driver.quit()

### Coletando Informações dos Jogos

In [100]:
for i in range(6):
    year = 2019 + i
    info_games = []
    
    for id in all_ids[i]:
        game_info_url = f"https://store.steampowered.com/api/appdetails?appids={id}&cc=us"
        
        request = requests.get(game_info_url, timeout=10)
        time.sleep(1)
        
        game_data = json.loads(request.text)
        
        if game_data is not None:
            
            # coletando: nome, data de lançamento, idade mínima, se é gratuito, tipo da moeda e preço.
            # caso queira coletar mais informações, atribuir variáveis abaixo.
            name = game_data[id]['data']['name']
            release_date = game_data[id]['data']['release_date']['date']
            
            min_age = game_data[id]['data']['required_age']
            is_free = game_data[id]['data']['is_free']
            
            
            # Se existir preco, atribua a variável preço
            # Caso contrário, atrivua a variável preço None
            if is_free == False:
                try:
                    preco = float(game_data[id]['data']['price_overview']['initial']) / 100
                except KeyError:
                    preco = None
            else:
                preco = None
                    
            # Criando dicionário do jogo atual e adicionando na lista
            info_games.append({
                'id' : id, 
                'game' : name, 
                'release_date' : release_date, 
                'min_age' : min_age, 
                'is_free' : is_free, 
                'price_USD' : preco
                })

    # criando arquivo csv
    df = pd.DataFrame(info_games)
    df.to_csv(f"most_played_{year}", index=False, encoding="utf-8")
    print(f"ARQUIVO most_player_{year}.csv CRIADO.")

ARQUIVO most_player_2019.csv CRIADO.
ARQUIVO most_player_2020.csv CRIADO.
ARQUIVO most_player_2021.csv CRIADO.
ARQUIVO most_player_2022.csv CRIADO.
ARQUIVO most_player_2023.csv CRIADO.
ARQUIVO most_player_2024.csv CRIADO.


### Visualizando Dados Coletados

In [109]:
df_years = []
for i in range(2019, 2025):
    df_years.append(pd.read_csv(f'most_played_{i}'))

df_years[5]

,id,game,release_date,min_age,is_free,price_USD
0,2358720,Black Myth: Wukong,"Aug 19, 2024",13,False,59.99
1,553850,HELLDIVERS™ 2,"Feb 8, 2024",17,False,39.99
2,578080,PUBG: BATTLEGROUNDS,"Dec 21, 2017",0,True,NaN
3,1245620,ELDEN RING,"Feb 24, 2022",16,False,59.99
4,730,Counter-Strike 2,"Aug 21, 2012",0,True,NaN
5,1623730,Palworld,"Jan 18, 2024",0,False,29.99
6,2767030,Marvel Rivals,"Dec 5, 2024",0,True,NaN
7,1172470,Apex Legends™,"Nov 4, 2020",0,True,NaN
8,2694490,Path of Exile 2,"Dec 6, 2024",0,False,29.99
9,570,Dota 2,"Jul 9, 2013",0,True,NaN
